# Population Forcast

In [32]:
import folium
import zipfile
import urllib.request
import pandas as pd
import geopandas as gpd
import numpy as np

In [33]:
# Read csv and rename the columns
population=pd.read_csv('../data/raw/population_2.csv',header=0,names=['S/T code','S/T name', 'GCCSA code', 'GCCSA name',\
                                                                      'SA4 code', 'SA4 name', 'SA3 code', 'SA3 name', 'SA2 code',\
                                                                      'SA2 name', '2001','2002', '2003','2004', '2005','2006',
                                                                      '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',\
                                                                      '2016','2017','2018', '2019','2020','2021','change 2011-2021 number','change 2011-2021 %',\
                                                                      'Area km2','population density 2021(persons/km2)'])

# Select the region of SA2 is Victoria
VIC_population = population.loc[population['S/T name']=='Victoria']
select_population = VIC_population.loc[:, ['S/T name', 'SA2 name', '2001','2002', '2003','2004', '2005','2006',\
                                           '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',\
                                           '2016','2017','2018', '2019','2020','2021','change 2011-2021 number','change 2011-2021 %',\
                                           'Area km2','population density 2021(persons/km2)']] 

In [34]:
VIC_population

,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,...,2016,2017,2018,2019,2020,2021,change 2011-2021 number,change 2011-2021 %,Area km2,population density 2021(persons/km2)
650,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011001,Alfredton,...,11852,12639,13520,14408,15472,16823,8371,99.0,52.7,319.2
651,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011002,Ballarat,...,12301,12261,12237,12311,12183,12076,-289,-2.3,12.4,975.5
652,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011005,Buninyong,...,7311,7407,7413,7452,7369,7232,507,7.5,51.6,140.2
653,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011006,Delacombe,...,7195,7617,8174,8876,9736,10640,4802,82.3,34.2,311.5
654,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011007,Smythes Creek,...,3990,4002,4040,4109,4148,4213,394,10.3,104.7,40.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21703,Colac - Corangamite,217031476,Otway,...,3556,3633,3707,3797,3905,3978,558,16.3,1505.9,2.6
1168,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,217041477,Moyne - East,...,6709,6714,6741,6790,6873,6989,210,3.1,3275.8,2.1
1169,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,217041478,Moyne - West,...,9603,9681,9773,9833,9842,9963,939,10.4,2037.8,4.9
1170,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,217041479,Warrnambool - North,...,21442,21676,21934,22157,22379,22462,2583,13.0,62.4,359.9


In [35]:
# Print out the name of the column to be calculated
col = select_population.columns.to_list()
columns = []
for i in range(len(col)):
    if col[i].startswith("20"):
        if col[i] != "2021":
            columns.append((col[i],col[i+1]))

In [36]:
# create function used to calculate the population growth rate in 20 years
def avg_year(year1, year2):
    result = 0
    if year1!=0:
       result =  year2/year1
    if year1 == 0:
        result = np.nan
    return result

In [37]:
# create a function that takes in two columns and spits out the average
df = pd.DataFrame()

for (col_1, col_2) in columns: 
    df['avg' + f'{col_1}' + f'{col_2}'] = select_population.apply(lambda x: avg_year(x[f"{col_1}"], x[f'{col_2}']), axis=1)

df['average change %'] = (df.sum(axis=1))/len(df.columns)
df

,avg20012002,avg20022003,avg20032004,avg20042005,avg20052006,avg20062007,avg20072008,avg20082009,avg20092010,avg20102011,...,avg20122013,avg20132014,avg20142015,avg20152016,avg20162017,avg20172018,avg20182019,avg20192020,avg20202021,average change %
650,1.058374,1.032994,1.029716,1.025926,1.016998,1.040379,1.033836,1.047030,1.036774,1.070687,...,1.072185,1.064237,1.067808,1.073648,1.066402,1.069705,1.065680,1.073848,1.087319,1.055274
651,1.018353,1.026221,1.014482,1.006563,1.007091,1.004208,1.005803,0.999679,0.998878,0.992216,...,0.999595,0.997976,0.997810,1.000081,0.996748,0.998043,1.006047,0.989603,0.991217,1.002498
652,1.014850,1.029265,1.011337,1.042171,1.030732,1.015571,1.019736,1.028631,1.025501,1.019712,...,1.018967,1.014032,1.015391,1.016688,1.013131,1.000810,1.005261,0.988862,0.981409,1.015562
653,1.017092,1.034556,1.021505,1.053527,1.071641,1.032732,1.027468,1.038886,1.025553,1.024390,...,1.041030,1.050423,1.039951,1.050979,1.058652,1.073126,1.085882,1.096890,1.092851,1.048416
654,1.018390,1.009769,1.018176,1.010078,1.009692,1.014681,1.017807,1.015309,1.016155,1.011924,...,1.010847,1.007920,1.005323,1.006051,1.003008,1.009495,1.017079,1.009491,1.015670,1.012037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,1.007822,1.009198,1.000000,0.994588,0.990550,1.008673,1.003439,0.996858,0.991404,0.988439,...,1.011863,1.006291,1.005399,1.005088,1.021654,1.020369,1.024278,1.028444,1.018694,1.007179
1168,0.997916,0.995823,0.995057,0.999247,1.002109,0.993085,1.003784,1.010858,1.005371,1.005936,...,0.997782,0.997777,0.997327,0.998958,1.000745,1.004021,1.007269,1.012224,1.016878,1.001998
1169,1.008416,1.007512,1.004379,1.003535,1.009863,1.010813,1.011272,1.009782,1.009800,1.006581,...,1.014509,1.008925,1.008952,1.014366,1.008122,1.009503,1.006139,1.000915,1.012294,1.009076
1170,1.023222,1.015875,1.011903,1.013101,1.019591,1.018836,1.012184,1.013712,1.013682,1.012478,...,1.017726,1.015428,1.013712,1.010605,1.010913,1.011903,1.010167,1.010019,1.003709,1.013879


In [38]:
# Add new column in to csv
select_population['population growth rate'] = df['average change %']
select_population

,S/T name,SA2 name,2001,2002,2003,2004,2005,2006,2007,2008,...,2017,2018,2019,2020,2021,change 2011-2021 number,change 2011-2021 %,Area km2,population density 2021(persons/km2),population growth rate
650,Victoria,Alfredton,5756,6092,6293,6480,6648,6761,7034,7272,...,12639,13520,14408,15472,16823,8371,99.0,52.7,319.2,1.055274
651,Victoria,Ballarat,11497,11708,12015,12189,12269,12356,12408,12480,...,12261,12237,12311,12183,12076,-289,-2.3,12.4,975.5,1.002498
652,Victoria,Buninyong,5320,5399,5557,5620,5857,6037,6131,6252,...,7407,7413,7452,7369,7232,507,7.5,51.6,140.2,1.015562
653,Victoria,Delacombe,4154,4225,4371,4465,4704,5041,5206,5349,...,7617,8174,8876,9736,10640,4802,82.3,34.2,311.5,1.048416
654,Victoria,Smythes Creek,3317,3378,3411,3473,3508,3542,3594,3658,...,4002,4040,4109,4148,4213,394,10.3,104.7,40.2,1.012037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,Victoria,Otway,3452,3479,3511,3511,3492,3459,3489,3501,...,3633,3707,3797,3905,3978,558,16.3,1505.9,2.6,1.007179
1168,Victoria,Moyne - East,6718,6704,6676,6643,6638,6652,6606,6631,...,6714,6741,6790,6873,6989,210,3.1,3275.8,2.1,1.001998
1169,Victoria,Moyne - West,8317,8387,8450,8487,8517,8601,8694,8792,...,9681,9773,9833,9842,9963,939,10.4,2037.8,4.9,1.009076
1170,Victoria,Warrnambool - North,17053,17449,17726,17937,18172,18528,18877,19107,...,21676,21934,22157,22379,22462,2583,13.0,62.4,359.9,1.013879


In [39]:
# Predicted population numbers for each of the three years.
select_population["2022"]=(select_population["2021"]*select_population['population growth rate']).round()
select_population["2023"]=(select_population["2022"]*select_population['population growth rate']).round()
select_population["2024"]=(select_population["2023"]*select_population['population growth rate']).round()
select_population["2025"]=(select_population["2024"]*select_population['population growth rate']).round()

# Calcuolate the population density between 2014 and 2025
select_population["population density 2014"]=(select_population["2014"]/select_population["Area km2"]).round()
select_population["population density 2015"]=(select_population["2015"]/select_population["Area km2"]).round()
select_population["population density 2016"]=(select_population["2016"]/select_population["Area km2"]).round()
select_population["population density 2017"]=(select_population["2017"]/select_population["Area km2"]).round()
select_population["population density 2018"]=(select_population["2018"]/select_population["Area km2"]).round()
select_population["population density 2019"]=(select_population["2019"]/select_population["Area km2"]).round()
select_population["population density 2020"]=(select_population["2020"]/select_population["Area km2"]).round()
select_population["population density 2021"]=(select_population["2021"]/select_population["Area km2"]).round()
select_population["population density 2022"]=(select_population["2022"]/select_population["Area km2"]).round()
select_population["population density 2023"]=(select_population["2023"]/select_population["Area km2"]).round()
select_population["population density 2024"]=(select_population["2024"]/select_population["Area km2"]).round()
select_population["population density 2025"]=(select_population["2025"]/select_population["Area km2"]).round()

In [40]:
# Save csv
select_population.to_csv('../data/curated/population_SA2.csv')

In [41]:
# Read the csv file of the converted poa and preprocess
postcode_SA2 = pd.read_csv('../data/raw/postcode_info.csv')
attribute = postcode_SA2.loc[:,['Postcode','SA2 NAME 2016','Latitude','Longitude']]
drop_dup = attribute.drop_duplicates(subset=['Postcode'])
rename = drop_dup.rename(columns={'SA2 NAME 2016':'SA2 name'})

In [42]:
# merge two csv files to convert SA2 to poa
merge_data = pd.DataFrame(pd.merge(rename,select_population,on='SA2 name', how='inner'))

In [43]:
merge_data

,Postcode,SA2 name,Latitude,Longitude,S/T name,2001,2002,2003,2004,2005,...,population density 2016,population density 2017,population density 2018,population density 2019,population density 2020,population density 2021,population density 2022,population density 2023,population density 2024,population density 2025
0,3002,East Melbourne,-37.818517,144.982207,Victoria,3731,3859,4243,4460,4495,...,1888.0,1895.0,1865.0,1867.0,1854.0,1711.0,1737.0,1762.0,1789.0,1815.0
1,8002,East Melbourne,-37.818517,144.982207,Victoria,3731,3859,4243,4460,4495,...,1888.0,1895.0,1865.0,1867.0,1854.0,1711.0,1737.0,1762.0,1789.0,1815.0
2,3005,Docklands,-37.824608,144.950858,Victoria,154,926,1913,3089,3779,...,4930.0,5801.0,6433.0,6823.0,6949.0,6642.0,9448.0,13437.0,19111.0,27182.0
3,3008,Docklands,-37.814719,144.948039,Victoria,154,926,1913,3089,3779,...,4930.0,5801.0,6433.0,6823.0,6949.0,6642.0,9448.0,13437.0,19111.0,27182.0
4,8012,Docklands,-37.814719,144.948039,Victoria,154,926,1913,3089,3779,...,4930.0,5801.0,6433.0,6823.0,6949.0,6642.0,9448.0,13437.0,19111.0,27182.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,3991,Wonthaggi - Inverloch,-38.483041,145.466781,Victoria,17647,17801,17956,18117,18250,...,31.0,32.0,33.0,34.0,35.0,37.0,37.0,38.0,39.0,40.0
594,3992,Wonthaggi - Inverloch,-38.569135,145.534315,Victoria,17647,17801,17956,18117,18250,...,31.0,32.0,33.0,34.0,35.0,37.0,37.0,38.0,39.0,40.0
595,3995,Wonthaggi - Inverloch,-38.578071,145.630713,Victoria,17647,17801,17956,18117,18250,...,31.0,32.0,33.0,34.0,35.0,37.0,37.0,38.0,39.0,40.0
596,3996,Wonthaggi - Inverloch,-38.629561,145.745609,Victoria,17647,17801,17956,18117,18250,...,31.0,32.0,33.0,34.0,35.0,37.0,37.0,38.0,39.0,40.0


In [44]:
# save csv file that is converted poa and drop blank rows
merge_data.dropna(axis='rows')
merge_data.to_csv('../data/curated/population_postcode.csv')

## Draw Geographic Map about population growth rate with SA2 region

In [45]:
sf = gpd.read_file("../data/raw/SA2_2021_AUST_GDA2020.shp")

In [46]:
sf = sf.rename(columns={'SA2_NAME21':'SA2 name'})
sf = sf.loc[sf['STE_NAME21'] == 'Victoria']

In [47]:
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [48]:
# Merge two dataframe and save as Geodataframe
result = gpd.GeoDataFrame(pd.merge(select_population[['SA2 name', 'population growth rate']], sf, on='SA2 name', how='inner'))

# create a JSON 
geoJSON = result[['SA2 name','geometry']].drop_duplicates('SA2 name').to_json()

# print the first 300 chars of the json
print(geoJSON[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"SA2 name": "Alfredton"}, "geometry": {"type": "Polygon", "coordinates": [[[143.78282104711133, -37.566657808073295], [143.75557764214773, -37.56346721632544], [143.7480171735439, -37.56259575584101], [143.73820


In [49]:
m = folium.Map(location=[-36, 145.3906], tiles="Stamen Terrain", zoom_start=7)

# refer to the folium documentations on more information on how to plot aggregated data
# SA2 level data
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=result, # data source
    columns=['SA2 name','population growth rate'], # the columns required
    key_on= 'properties.SA2 name', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='population growth rate in 20 years'
)

c.add_to(m)
m.save('../plots/Population growth rate map.html')